DATA IMPORT AND CLEANUP

Step 1: Import movies_metadata, do some minor cleanup and data typing 

In [50]:
import pandas as pd
import numpy as np
import json
from json import JSONDecodeError

df_metadata = pd.read_csv("./data/movies_metadata.csv", dtype=str)

# Create has_homepage column
df_metadata['has_homepage'] = np.where(df_metadata['homepage'].isnull(), 0, 1 )
# drop uneeded columns
df_metadata = df_metadata.drop(['homepage', 'overview', 'poster_path', 'original_title', 'status', 'tagline', 'title', 'video'], axis=1)


# Fill the belongs_to_collection values with a 0 or 1
df_metadata['belongs_to_collection'] = df_metadata['belongs_to_collection'].fillna(0)
df_metadata.loc[df_metadata['belongs_to_collection'] != 0, 'belongs_to_collection'] = 1

# Cast belongs_to_collection to int
df_metadata = df_metadata.astype({'belongs_to_collection' : int})

# Eliminate erroneous columns (where id is not an int)
df_metadata = df_metadata[df_metadata['id'].apply(lambda x: str(x).isdigit())]
# cast id to int
df_metadata = df_metadata.astype({'id' : int})


df_metadata.head()

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,vote_average,vote_count,has_homepage
0,False,1,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,en,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",7.7,5415,1
1,False,0,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,en,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",6.9,2413,0
2,False,1,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,en,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",6.5,92,0
3,False,0,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,en,3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",6.1,34,0
4,False,1,0,"[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,en,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",5.7,173,0









Step 2: Create one-hot encodings for each movie Genre

In [51]:
# Change genres column into a list of json (dict) objects 
df_metadata['genres'] = df_metadata['genres'].apply(lambda x: json.loads(x.replace('\'', '"')))

# Find all genre types
Genres = dict()
def checkGenre(listObj):
    for obj in listObj:
        if obj['name'] not in Genres:
            Genres[obj['name']] = 0
        Genres[obj['name']] += 1
    return listObj
        
     
df_metadata['genres'].apply(checkGenre)


for gen in Genres:
    df_metadata[gen] = 0

def fillGenres(row):
    for obj in row['genres']:
        row[obj['name']] = 1
    return row
    
df_metadata = df_metadata.apply(fillGenres, axis=1)

# drop genres column now that data has been flattened
df_metadata = df_metadata.drop(['genres'], axis=1)


df_metadata[['id'] + list(Genres.keys())].head()


,id,Animation,Comedy,Family,Adventure,Fantasy,Romance,Drama,Action,Crime,...,Horror,History,Science Fiction,Mystery,War,Foreign,Music,Documentary,Western,TV Movie
0,862,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8844,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15602,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,31357,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11862,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0









Step 3: Create one-hot encodings for the major production companies 

In [52]:
# Change production_companies column into a list of json (dict) objects 
#clean up

ProdCompanies = dict()
def tester(val):
    try:
        compList = json.loads(val)
    except JSONDecodeError as e:
        return
       
    for entry in compList:
        if entry['name'] not in ProdCompanies:
            ProdCompanies[entry['name']] = 0
        ProdCompanies[entry['name']] +=1
    
   
              
        
    
# correct for nan values in data, get into json object form
df_metadata['production_companies'] = df_metadata['production_companies'].fillna('[]')
df_metadata['production_companies'] = df_metadata['production_companies'].apply(lambda x: str(x).replace('\'', '"'))
df_metadata['production_companies'].apply(tester)

print(f'Most Common Production Companies:')
print(sorted(ProdCompanies.items(), key= lambda item: item[1], reverse=True)[:10])

# create categories for major film studios
df_metadata['Warner Bros'] = 0
df_metadata['MGM'] = 0
df_metadata['Paramount Pictures'] = 0
df_metadata['Universal'] = 0
df_metadata['20th Century Fox'] = 0
df_metadata['Disney Pixar'] = 0
df_metadata['Columbia Pictures'] = 0

df_metadata['Number of Production Orgs'] = 0


def fillProductions(row):
    try:
        compList = json.loads(row['production_companies'])
    except JSONDecodeError as e:
        return row
    
    countr = 0
    for entry in compList: 
        countr += 1
        
        if entry['name'] == 'Warner Bros.':
            row['Warner Bros'] += 1
        elif entry['name'] == 'Metro-Goldwyn-Mayer (MGM)':
            row['MGM'] += 1
        elif entry['name'] == 'Paramount Pictures':
            row['Paramount Pictures'] += 1
        elif entry['name'] == 'Universal Pictures':
            row['Universal'] += 1
        elif entry['name'] == 'Twentieth Century Fox Film Corporation':
            row['20th Century Fox'] += 1
        elif entry['name'] in ['Columbia Pictures Corporation','Columbia Pictures']:
            row['Columbia Pictures'] += 1
        elif entry['name'] in ['Walt Disney Productions', 'Walt Disney Animation Studios', 'Pixar Animation Studios']:
            row['Disney Pixar'] += 1
      
    row['Num_Production_Orgs'] = countr
    
    return row

df_metadata = df_metadata.apply(fillProductions, axis=1)

# drop companies field
df_metadata = df_metadata.drop(['production_companies'], axis=1)
df_metadata[['id', 'Num_Production_Orgs', 'Warner Bros','MGM', 'Paramount Pictures', 'Universal', '20th Century Fox','Columbia Pictures', 'Disney Pixar' ]].head()


Most Common Production Companies:
[('Warner Bros.', 1243), ('Metro-Goldwyn-Mayer (MGM)', 1045), ('Paramount Pictures', 1000), ('Universal Pictures', 826), ('Twentieth Century Fox Film Corporation', 821), ('Columbia Pictures Corporation', 446), ('Columbia Pictures', 429), ('Canal+', 390), ('RKO Radio Pictures', 290), ('United Artists', 277)]


,id,Num_Production_Orgs,Warner Bros,MGM,Paramount Pictures,Universal,20th Century Fox,Columbia Pictures,Disney Pixar
0,862,1.0,0,0,0,0,0,0,1
1,8844,3.0,0,0,0,0,0,0,0
2,15602,2.0,1,0,0,0,0,0,0
3,31357,1.0,0,0,0,0,1,0,0
4,11862,2.0,0,0,0,0,0,0,0








Step 4: Extract year and month as values from release_date

In [53]:
# extract out the release year and month 
df_metadata['year'] = 0
df_metadata['month'] = 0

def extractDate(row):
    
    if type(row['release_date']) == float:
        row['year'] = 0
        row['month'] = 0
        return row
        
    dateString = row['release_date']
    dateList = dateString.split('-')
    row['year'] = int(dateList[0])
    row['month'] = int(dateList[1])
     
    return row

df_metadata = df_metadata.apply(extractDate, axis=1)

# drop date column
df_metadata = df_metadata.drop(['release_date'], axis=1)

df_metadata[['id', 'year', 'month']].head()



,id,year,month
0,862,1995,10
1,8844,1995,12
2,15602,1995,12
3,31357,1995,12
4,11862,1995,2









Step 5: Create features for number of countries involved in production, and binary feature if the movie is US-made

In [54]:
# Extract Number of Production companies

df_metadata['Num_Prod_Countries'] = 0
df_metadata['Made_in_US'] = 0

def ExtractCountries(row):
    if row['production_countries'] == 'nan':
        return row
    
    countriesString = row['production_countries'].replace('\'', '"')
    countriesDict = json.loads(countriesString)
    countr = 0
    for entry in countriesDict:
        countr +=1
        if entry['iso_3166_1'] == 'US':
            row['Made_in_US'] = 1
        
    row['Num_Prod_Countries'] = countr
    return row
   
        

df_metadata['production_countries'] = df_metadata['production_countries'].apply(lambda x: str(x).replace('D\'Ivoire', 'DIvoire'))
df_metadata['production_countries'] = df_metadata['production_countries'].apply(lambda x: str(x).replace('ople\'s Dem', 'oples Dem'))
df_metadata = df_metadata.apply(ExtractCountries, axis=1)

df_metadata = df_metadata.drop(['production_countries'], axis=1)

df_metadata[['id', 'Made_in_US', 'Num_Prod_Countries']].head()




,id,Made_in_US,Num_Prod_Countries
0,862,1,1
1,8844,1,1
2,15602,1,1
3,31357,1,1
4,11862,1,1








Step 6: Create feature for number of languaegs spoken in the movie

In [55]:
# extract number of spoken languages

def getLangs(val):
    if str(val) == 'nan':
        return 0
   
    val = str(val).replace('\'', '"')
    val = val.replace('\\x', '')
    valList = json.loads(val)
    countr = 0
    for entry in valList:
        countr +=1
        
    return countr
    
    
df_metadata['spoken_languages'] = df_metadata['spoken_languages'].apply(getLangs)

df_metadata[['id', 'spoken_languages']].head()


print(df_metadata.columns)

Index(['20th Century Fox', 'Action', 'Adventure', 'Animation',
       'Columbia Pictures', 'Comedy', 'Crime', 'Disney Pixar', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'MGM',
       'Music', 'Mystery', 'Num_Production_Orgs', 'Number of Production Orgs',
       'Paramount Pictures', 'Romance', 'Science Fiction', 'TV Movie',
       'Thriller', 'Universal', 'War', 'Warner Bros', 'Western', 'adult',
       'belongs_to_collection', 'budget', 'has_homepage', 'id', 'imdb_id',
       'original_language', 'popularity', 'revenue', 'runtime',
       'spoken_languages', 'vote_average', 'vote_count', 'year', 'month',
       'Num_Prod_Countries', 'Made_in_US'],
      dtype='object')








Step 7: Merge with with credits data from credits.csv 

In [56]:
# Pull in next Data file
df_credits = pd.read_csv("./data/credits.csv", dtype=str)

df_metadata = df_metadata.drop_duplicates(subset=['id'])
df_credits = df_credits.drop_duplicates(subset=['id'])

df_credits = df_credits.astype({'id' : int})
df = df_metadata.merge(df_credits, on='id', how='inner')

df.head()





,20th Century Fox,Action,Adventure,Animation,Columbia Pictures,Comedy,Crime,Disney Pixar,Documentary,Drama,...,runtime,spoken_languages,vote_average,vote_count,year,month,Num_Prod_Countries,Made_in_US,cast,crew
0,0,0,0,1,0,1,0,1,0,0,...,81.0,1,7.7,5415,1995,10,1,1,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,0,0,1,0,0,0,0,0,0,0,...,104.0,2,6.9,2413,1995,12,1,1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,0,0,0,0,0,1,0,0,0,0,...,101.0,1,6.5,92,1995,12,1,1,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,1,0,0,0,0,1,0,0,0,1,...,127.0,1,6.1,34,1995,12,1,1,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,0,0,0,0,0,1,0,0,0,0,...,106.0,1,5.7,173,1995,2,1,1,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


Step 8: Find most common actors to make A-list

In [57]:
actorDict = dict()
# keep track of length to cast size ratio (for use in filling in missing values)
CastToStrLenRatio = []

def gatherActors(val):
    # make json-ready
    global CastToStrLenRatio
    
    strLen = len(val)
    try:
        # get rid of null values
        val = str(val).replace('None', '"sample"')
        # set keys to double quotes
        val = str(val).replace('\'cast_id\'', '"cast_id"')
        val = str(val).replace('\'character\'', '"character"')
        val = str(val).replace('\'credit_id\'', '"credit_id"')
        val = str(val).replace('\'gender\'', '"gender"')
        val = str(val).replace('\'name\'', '"name"')
        val = str(val).replace('\'order\'', '"order"')
        val = str(val).replace('\'profile_path\'', '"profile_path"')
        val = str(val).replace('\'id\'', '"id"')
        
        # eliminiate situtions like: "character": '"Nickname"'
        val = str(val).replace('\'"', '\'')
        val = str(val).replace('"\'', '\'')
        
        # set single quotes around each key's value
        val = str(val).replace(': \'', ': "')
        val = str(val).replace('\', ', '", ')
        val = str(val).replace('\'}, {', '"}, {')
        val = str(val).replace('\'}]', '"}]')
        val = str(val).replace('\'', '')
        
        castDict = json.loads(val)
        for entry in castDict:
            if entry['id'] not in actorDict:
                actorDict[entry['id']] = dict()
                actorDict[entry['id']]['name'] = entry['name']
                actorDict[entry['id']]['roles'] = 0
            actorDict[entry['id']]['roles'] += 1
            
        # keep track of string length to cast size ratio (for use in filling in missing values)
        CastToStrLenRatio.append(len(castDict) / strLen)
        
    # some errors in JSON are unfixable
    except JSONDecodeError as e: 
      return val 

    return val
        

df['cast'].apply(gatherActors)    

# Collect id's of A-List actors (top 25 most common in dataset)
AListActors = [int(x[0]) for x in sorted(actorDict.items(), key= lambda item: item[1]['roles'], reverse=True)[:25] ]

print('A-List Actors:')
for idd in AListActors:
    name = actorDict[idd]['name']
    print(f'{idd}: {name}')
    
    
# Add dataframe column for has_a_list_actor
df['Has_AList_Actor'] = 0
df['Size_of_Cast'] = 0

# calculate average cast string length to cast size ratio (for use in filling in missing values)
ratios = np.array(CastToStrLenRatio)
CastToStrLenRatioVal = pd.DataFrame(data=ratios, columns=['ratios'])['ratios'].mean()


def checkActors(row):
  
    global AListActors
    val = row['cast']
    try:
        # get rid of null values
        val = str(val).replace('None', '"sample"')
        # set keys to double quotes
        val = str(val).replace('\'cast_id\'', '"cast_id"')
        val = str(val).replace('\'character\'', '"character"')
        val = str(val).replace('\'credit_id\'', '"credit_id"')
        val = str(val).replace('\'gender\'', '"gender"')
        val = str(val).replace('\'name\'', '"name"')
        val = str(val).replace('\'order\'', '"order"')
        val = str(val).replace('\'profile_path\'', '"profile_path"')
        val = str(val).replace('\'id\'', '"id"')
        
        # eliminiate situtions like: "character": '"Nickname"'
        val = str(val).replace('\'"', '\'')
        val = str(val).replace('"\'', '\'')
        
        # set double quotes around each key's value
        val = str(val).replace(': \'', ': "')
        val = str(val).replace('\', ', '", ')
        val = str(val).replace('\'}, {', '"}, {')
        val = str(val).replace('\'}]', '"}]')
        val = str(val).replace('\'', '')
        
        castDict = json.loads(val)
        row['Size_of_Cast'] = len(castDict)
        for entry in castDict:
            if entry['id'] in AListActors:
                row['Has_AList_Actor'] = 1
                
            
    # some errors in JSON are unfixable
    except JSONDecodeError as e: 
        row['Size_of_Cast'] = CastToStrLenRatioVal * len(row['cast'])
        return row 


    return row
    
    
df = df.apply(checkActors, axis=1)
df = df.drop(['cast'], axis=1)

df.head(6)

A-List Actors:
121323: Bess Flowers
113: Christopher Lee
4165: John Wayne
2231: Samuel L. Jackson
18897: Jackie Chan
16927: Gérard Depardieu
8516: John Carradine
3895: Michael Caine
15831: Frank Welker
55636: Donald Sutherland
11160: Danny Trejo
380: Robert De Niro
884: Steve Buscemi
2047: Danny Glover
30530: Irving Bacon
1230: John Goodman
5049: John Hurt
2778: Dennis Hopper
62: Bruce Willis
4038: Susan Sarandon
4690: Christopher Walken
56890: Malcolm McDowell
21315: Eric Roberts
5293: Willem Dafoe
9221: Donald Pleasence


,20th Century Fox,Action,Adventure,Animation,Columbia Pictures,Comedy,Crime,Disney Pixar,Documentary,Drama,...,spoken_languages,vote_average,vote_count,year,month,Num_Prod_Countries,Made_in_US,crew,Has_AList_Actor,Size_of_Cast
0,0,0,0,1,0,1,0,1,0,0,...,1,7.7,5415,1995,10,1,1,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",0,13.0
1,0,0,1,0,0,0,0,0,0,0,...,2,6.9,2413,1995,12,1,1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",0,26.0
2,0,0,0,0,0,1,0,0,0,0,...,1,6.5,92,1995,12,1,1,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",0,7.0
3,1,0,0,0,0,1,0,0,0,1,...,1,6.1,34,1995,12,1,1,"[{'credit_id': '52fe44779251416c91011acb', 'de...",0,10.0
4,0,0,0,0,0,1,0,0,0,0,...,1,5.7,173,1995,2,1,1,"[{'credit_id': '52fe44959251416c75039ed7', 'de...",0,12.0
5,0,1,0,0,0,0,1,0,0,1,...,2,7.7,1886,1995,12,1,1,"[{'credit_id': '52fe4292c3a36847f802916d', 'de...",1,65.0


Step 9: Extract features from the crew list 

In [58]:
directorDict = dict()
CrewToStrLenRatio = []


def gatherDirectors(val):
   
    global CrewToStrLenRatio
    strLen = len(val)
    
    try:
        # double quotes around key values
        val = str(val).replace('\'credit_id\'', '"credit_id"')
        val = str(val).replace('\'department\'', '"department"')
        val = str(val).replace('\'gender\'', '"gender"')
        val = str(val).replace('\'id\'', '"id"')
        val = str(val).replace('\'job\'', '"job"')
        val = str(val).replace('\'name\'', '"name"')
        val = str(val).replace('\'profile_path\'', '"profile_path"')
        
        # double quotes around value values
        val = str(val).replace(': \'', ': "')
        val = str(val).replace('\', ', '", ')
        val = str(val).replace('\'}, {', '"}, {')
        val = str(val).replace('\'}]', '"}]')
        
        # get rid of all reamining single quotes
        val = str(val).replace('\'', '')
       
        val = str(val).replace('None', '"sample"')
        crewString = json.loads(val)
        for entry in crewString:
            # if the role is director 
            if entry['department'] == 'Directing' and entry['job'] == 'Director':
                if entry['id'] not in directorDict:
                    directorDict[entry['id']] = dict()
                    directorDict[entry['id']]['name'] = entry['name']
                    directorDict[entry['id']]['count'] = 0
                directorDict[entry['id']]['count'] +=1
    
        CrewToStrLenRatio.append(len(crewString) / strLen)
 
    # formatting situation that cannot be avoided
    except JSONDecodeError as e:
        return val

    return val
    


df['crew'].apply(gatherDirectors)

# Collect id's of directors (top 25 most common in dataset)
topDirectors = [int(x[0]) for x in sorted(directorDict.items(), key= lambda item: item[1]['count'], reverse=True)[:25] ]


print('Top Directors:')
for idd in topDirectors:
    name = directorDict[idd]['name']
    count = directorDict[idd]['count']
    print(f'{idd}: {name} ({count})')
    
    
ratios = np.array(CrewToStrLenRatio)
CrewToStrLenRatioVal = pd.DataFrame(data=ratios, columns=['ratios'])['ratios'].mean()


# Add dataframe columns
df['Has_Top_Director'] = 0
df['Size_of_Crew'] = 0


def checkCrew(row):
    global topDirectors
    val = row['crew']
    
    strLen = len(val)
    
    try:
        # double quotes around key values
        val = str(val).replace('\'credit_id\'', '"credit_id"')
        val = str(val).replace('\'department\'', '"department"')
        val = str(val).replace('\'gender\'', '"gender"')
        val = str(val).replace('\'id\'', '"id"')
        val = str(val).replace('\'job\'', '"job"')
        val = str(val).replace('\'name\'', '"name"')
        val = str(val).replace('\'profile_path\'', '"profile_path"')
        
        # double quotes around value values
        val = str(val).replace(': \'', ': "')
        val = str(val).replace('\', ', '", ')
        val = str(val).replace('\'}, {', '"}, {')
        val = str(val).replace('\'}]', '"}]')
        
        # get rid of all reamining single quotes
        val = str(val).replace('\'', '')
       
        val = str(val).replace('None', '"sample"')
        crewString = json.loads(val)
        for entry in crewString:
             if entry['department'] == 'Directing' and entry['job'] == 'Director' and entry['id'] in topDirectors:
                    row['Has_Top_Director'] = 1
                    
        row['Size_of_Crew'] = len(crewString)
     
 
    # formatting situation that cannot be avoided
    except JSONDecodeError as e:
        row['Size_of_Crew'] = CrewToStrLenRatioVal * strLen
        return row

    return row
    
    

df = df.apply(checkCrew, axis=1)
df = df.drop(['crew'], axis=1)
df.head(10)


Top Directors:
8500: John Ford (68)
4109: Michael Curtiz (65)
6818: Werner Herzog (55)
2636: Alfred Hitchcock (53)
11523: Georges Méliès (51)
3776: Jean-Luc Godard (50)
1243: Woody Allen (49)
39996: Sidney Lumet (46)
13848: Charlie Chaplin (44)
14643: William A. Wellman (43)
37360: Henry Hathaway (43)
8823: Raoul Walsh (43)
70862: Richard Thorpe (42)
1032: Martin Scorsese (41)
14674: George Cukor (39)
6593: John Huston (39)
6648: Ingmar Bergman (39)
9789: Robert Altman (38)
68: Fritz Lang (38)
9054: Mervyn LeRoy (38)
19069: Claude Chabrol (37)
1744: Robert Wise (37)
17282: Takashi Miike (37)
5281: Spike Lee (36)
2662: Frank Capra (36)


,20th Century Fox,Action,Adventure,Animation,Columbia Pictures,Comedy,Crime,Disney Pixar,Documentary,Drama,...,vote_average,vote_count,year,month,Num_Prod_Countries,Made_in_US,Has_AList_Actor,Size_of_Cast,Has_Top_Director,Size_of_Crew
0,0,0,0,1,0,1,0,1,0,0,...,7.7,5415,1995,10,1,1,0,13.0,0,106.0
1,0,0,1,0,0,0,0,0,0,0,...,6.9,2413,1995,12,1,1,0,26.0,0,16.0
2,0,0,0,0,0,1,0,0,0,0,...,6.5,92,1995,12,1,1,0,7.0,0,4.0
3,1,0,0,0,0,1,0,0,0,1,...,6.1,34,1995,12,1,1,0,10.0,0,10.0
4,0,0,0,0,0,1,0,0,0,0,...,5.7,173,1995,2,1,1,0,12.0,0,7.0
5,0,1,0,0,0,0,1,0,0,1,...,7.7,1886,1995,12,1,1,1,65.0,0,71.0
6,0,0,0,0,0,1,0,0,0,0,...,6.2,141,1995,12,2,1,0,57.0,0,53.0
7,0,1,1,0,0,0,0,0,0,1,...,5.4,45,1995,12,1,1,0,7.0,0,4.0
8,0,1,1,0,0,0,0,0,0,0,...,5.5,174,1995,12,1,1,0,6.0,0,9.0
9,0,1,1,0,0,0,0,0,0,0,...,6.6,1194,1995,11,2,1,0,20.0,0,46.0


Step 10: Collect user rating data

In [59]:
# import rating data
df_ratings = pd.read_csv("./data/ratings.csv", dtype=str)

# drop uneeded columns
df_ratings = df_ratings.drop(['timestamp','userId'], axis=1)

# Cast movieId to int
df_ratings = df_ratings.astype({'movieId' : int})

# Cast rating to int
df_ratings = df_ratings.astype({'rating' : float})

# Average rating value for each movie
df_ratings = df_ratings.groupby(['movieId'], as_index=False).mean()

df_ratings

,movieId,rating
0,1,3.888157
1,2,3.236953
2,3,3.175550
3,4,2.875713
4,5,3.079565
...,...,...
45110,176267,4.000000
45111,176269,3.500000
45112,176271,5.000000
45113,176273,1.000000


Step 11: Minor edits with vote count

In [60]:
# see columns
df = df.drop(['imdb_id', 'original_language'], axis=1)

# fill in nulls for vote count with 0
df['vote_count'] = df['vote_count'].fillna(0)
# drop movies with no ratings 
df = df[df['vote_count'].apply(lambda x: int(x) > 0)]


Step 12: fill in missing budget and revenue values